# Modeling Your Data - Lab

## Introduction 

In this lab you'll perform a full linear regression on the data. You'll implement the process demonstrated in the previous lesson, taking a stepwise approach to analyze and improve the model along the way.

## Objectives
You will be able to:

* Remove predictors with p-values too high and refit the model
* Examine and interpret the model results
* Split data into training and testing sets
* Fit a regression model to the data set using statsmodel library


## Build an Initial Regression Model

To start, perform a train-test split and create an initial regression model to model the `list_price` using all of your available features.

> **Note:** In order to write the model you'll have to do some tedious manipulation of your column names. Statsmodels will not allow you to have spaces, apostrophe or arithmetic symbols (+) in your column names. Preview them and refine them as you go.  
**If you receive an error such as "PatsyError: error tokenizing input (maybe an unclosed string?)" then you need to further preprocess your column names.**

In [ ]:
import pandas as pd
import numpy as np 

from statsmodels.formula.api import ols
import scipy.stats as stats
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt


In [ ]:
df = pd.read_csv('Lego_dataset_cleaned.csv')
print(list(df.columns)[0:10])
len(df.columns)

In [ ]:
#Your code here
def col_format(df):
    corrected_cols=[]
    for i in df.columns:
        i=i.replace(' ', '_')
        i=i.replace('-', '_')
        i=i.replace('+', 'plus')
        i=i.replace("'", "")
        i=i.replace('.', '')
        i=i.replace('™', '')
        i=i.replace('®', '')
        i=i.replace('½', 'half')
        corrected_cols.append(i)
    return corrected_cols
col_format(df)[:10]

In [ ]:
df.columns=col_format(df)
df.columns

In [ ]:
# subs = [(' ', '_'),('.',''),("'",""),('™', ''), ('®',''),('+','plus'), ('½','half'), ('-','_')]
# def col_formatting(col):
#     for old, new in subs:
#         col = col.replace(old,new)
#     return col

# df.columns = [col_formatting(col) for col in df.columns]

In [ ]:
train, test = train_test_split(df, random_state=42)

## Remove the Uninfluential Features

Based on the initial model, remove those features which do not appear to be statistically relevant and rerun the model.

In [ ]:
#Your code here

predictors=list(df.drop(labels='list_price', axis=1).columns)
predictors= '+'.join(predictors)
outcome='list_price'
formula=outcome + '~' + predictors
model = ols(formula=formula, data=train).fit()
model.summary()


> **Comment:** You should see that the model performance is identical. Additionally, observe that there are further features which have been identified as unimpactful. Continue to refine the model accordingly.

In [ ]:
summary = model.summary()
p_table = summary.tables[1]
p_table = pd.DataFrame(p_table.data)
p_table.columns = p_table.iloc[0]
p_table = p_table.drop(0)
p_table = p_table.set_index(p_table.columns[0])
p_table['P>|t|'] = p_table['P>|t|'].astype(float)
x = list(p_table[p_table['P>|t|']<0.05].index)
x.remove('Intercept')

In [ ]:
#Refit model with subset features
predictors = '+'.join(x)
formula = outcome + "~" + predictors
model = ols(formula=formula, data=train).fit()
model.summary()

In [ ]:
#Extract the p-value table from the summary and use it to subset our features
summary = model.summary()

p_table = summary.tables[1]
p_table = pd.DataFrame(p_table.data)
p_table.columns = p_table.iloc[0]
p_table = p_table.drop(0)
p_table = p_table.set_index(p_table.columns[0])
p_table['P>|t|'] = p_table['P>|t|'].astype(float)
x = list(p_table[p_table['P>|t|']<0.05].index)
x.remove('Intercept')
p_table.head()

In [ ]:
#Refit model with subset features
predictors = '+'.join(x)
formula = outcome + "~" + predictors
model = ols(formula=formula, data=train).fit()
model.summary()

## Investigate Multicollinearity

There are still a lot of features in the current model! Chances are there are some strong multicollinearity issues. Begin to investigate the extend of this problem.

In [ ]:
#Your code here
from statsmodels.stats.outliers_influence import variance_inflation_factor
X = df[x]
vif = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
list(zip(x, vif))[:5]

## Perform Another Round of Feature Selection

Once again, subset your features based on your findings above. Then rerun the model once again.

In [ ]:
#Your code here
vif_scores = list(zip(x, vif))
x = [x for x,vif in vif_scores if vif < 5]
print(len(vif_scores), len(x_cols))

In [ ]:
predictors = '+'.join(x)
formula = outcome + "~" + predictors
model = ols(formula=formula, data=train).fit()
model.summary()

## Check the Normality Assumption

Check whether the normality assumption holds for your model.

In [ ]:
# Your code here
import statsmodels.api as sm
fig = sm.graphics.qqplot(model.resid, dist=stats.norm, line='45', fit=True)


## Check Homoscedasticity Assumption

Check whether the model's errors are indeed homoscedastic or if they violate this principle and display heteroscedasticity.

In [ ]:
#Your code here
plt.scatter(model.predict(train[x]), model.resid)
plt.plot(model.predict(train[x]), [0 for i in range(len(train))])

> **Comment:** This displays a fairly pronounced 'funnel' shape: errors appear to increase as the list_price increases. This doesn't bode well for our model. Subsetting the data to remove outliers and confiding the model to this restricted domain may be necessary. A log transformation or something equivalent may also be appropriate.

## Make Additional Refinements

From here, make additional refinements to your model based on the above analysis. As you progress, continue to go back and check the assumptions for the updated model. Be sure to attempt at least 2 additional model refinements.

> **Comment:** Based on the above plots, it seems as though outliers are having a substantial impact on the model. As such, removing outliers may be appropriate. Investigating the impact of a log transformation is also worthwhile.

In [ ]:
#Your code here
df.list_price.hist()


In [ ]:
for i in range(80,100):
    q = i/100
    print("{} percentile: {}".format(q, df.list_price.quantile(q=q)))


In [ ]:
orig_tot = len(df)
df = df[df.list_price < 450] #Subsetting to remove extreme outliers
print('Percent removed:', (orig_tot -len(df))/orig_tot)
df.list_price = df.list_price.map(np.log) #Applying a log transformation
train, test = train_test_split(df)

#Refit model with subset features
predictors = '+'.join(x)
formula = outcome + "~" + predictors
model = ols(formula=formula, data=train).fit()
model.summary()

In [ ]:
fig = sm.graphics.qqplot(model.resid, dist=stats.norm, line='45', fit=True)


In [ ]:
# Comment: There is still a clear underestimate now by the model as the log transformed 
# list price increases, but the model is much improved. Further subsetting is potentially warranted.
plt.scatter(model.predict(train[x]), model.resid)
plt.plot(model.predict(train[x]), [0 for i in range(len(train))])

## Summary

Well done! As you can see, regression can be a challenging task that requires you to make decisions along the way, try alternative approaches and make ongoing refinements. These choices depend on the context and specific use cases. 